## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [1]:
import numpy as np
import pandas as pd

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 69

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯ'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [2]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('../kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [3]:
import nltk
from nltk import word_tokenize

import re

WORD = re.compile(r'\w+')

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

def find_max_str(companies):
    max_str = ''
    for company in companies:
        for word in WORD.findall(company):
            if(has_letters(word)):
                max_str = word if len(max_str) < len(word) else max_str 
    return max_str

def find_index_of_substring(string, companies):
    for i in range(len(companies)):
        if(companies[i].find(string) != -1):
            return i

В данной строке артефакт, подробнее в 'Testing.ipynb'

In [4]:
del train_companies[471970]

In [13]:
def extract_features(company, test=False):
    
    # Длина слова, количество гласных/согласных, положение в слове,
    # есть ли в слове что-то помимо букв и то же самое для предыдущего слова
    num_of_features = 10
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in WORD.findall(company):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    for word in WORD.findall(company):
        if(has_letters(word)):
            if(index > 0):
                features[index][5:] = features[index - 1][:5]
            features[index][0] = index/size
            features[index][1] = len(word)/MAX_WORD
            features[index][2] = count_consonants(word)/len(word)
            features[index][3] = count_vowels(word)/len(word)
            features[index][4] = int(has_not_letters(word))
            if(word[0].isupper()):
                if(word[-1].isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return list(features), list(labels)
    else:
        return list(features)

In [14]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features += curr_features
            labels += curr_labels
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [15]:
def transform(company, labels):
    corrected = company
    index = 0
    for word in WORD.findall(company):
        if (has_letters(word)):
            print(labels[index])
            if (labels[index] == 0):
                if(len(word) < 4):
                    corrected = corrected.replace(' ' + word + ' ', ' ' + word.lower() + ' ')
                else:
                    corrected = corrected.replace(word, word.lower())
            elif(labels[index] == 1):
                if(len(word) < 4):
                    corrected = corrected.replace(' ' + word + ' ', ' ' + word[0] + word[1:].lower() + ' ')
                else:
                    corrected = corrected.replace(word, word[0] + word[1:].lower())
            index += 1
    return corrected

In [16]:
X_train, y_train = generate_dataset(train_companies[:100000])

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier()
clf.fit(X_train, y_train)

Learning rate set to 0.109939
0:	learn: 0.9850376	total: 231ms	remaining: 3m 50s
1:	learn: 0.8991012	total: 388ms	remaining: 3m 13s
2:	learn: 0.8328286	total: 549ms	remaining: 3m 2s
3:	learn: 0.7757945	total: 701ms	remaining: 2m 54s
4:	learn: 0.7311351	total: 861ms	remaining: 2m 51s
5:	learn: 0.6908427	total: 1.02s	remaining: 2m 49s
6:	learn: 0.6574303	total: 1.2s	remaining: 2m 49s
7:	learn: 0.6292450	total: 1.35s	remaining: 2m 47s
8:	learn: 0.6052306	total: 1.52s	remaining: 2m 47s
9:	learn: 0.5833743	total: 1.68s	remaining: 2m 46s
10:	learn: 0.5649259	total: 1.83s	remaining: 2m 44s
11:	learn: 0.5466810	total: 1.99s	remaining: 2m 43s
12:	learn: 0.5289889	total: 2.14s	remaining: 2m 42s
13:	learn: 0.5152889	total: 2.3s	remaining: 2m 41s
14:	learn: 0.5011413	total: 2.46s	remaining: 2m 41s
15:	learn: 0.4898325	total: 2.6s	remaining: 2m 40s
16:	learn: 0.4789927	total: 2.76s	remaining: 2m 39s
17:	learn: 0.4689561	total: 2.9s	remaining: 2m 38s
18:	learn: 0.4607996	total: 3.06s	remaining: 2m 3

157:	learn: 0.3017331	total: 25s	remaining: 2m 13s
158:	learn: 0.3015213	total: 25.2s	remaining: 2m 13s
159:	learn: 0.3013085	total: 25.3s	remaining: 2m 12s
160:	learn: 0.3011552	total: 25.5s	remaining: 2m 12s
161:	learn: 0.3009598	total: 25.6s	remaining: 2m 12s
162:	learn: 0.3005966	total: 25.8s	remaining: 2m 12s
163:	learn: 0.3003553	total: 25.9s	remaining: 2m 12s
164:	learn: 0.3002016	total: 26.1s	remaining: 2m 12s
165:	learn: 0.2999207	total: 26.2s	remaining: 2m 11s
166:	learn: 0.2995965	total: 26.4s	remaining: 2m 11s
167:	learn: 0.2994833	total: 26.5s	remaining: 2m 11s
168:	learn: 0.2991968	total: 26.7s	remaining: 2m 11s
169:	learn: 0.2989519	total: 26.8s	remaining: 2m 11s
170:	learn: 0.2987068	total: 27s	remaining: 2m 10s
171:	learn: 0.2985669	total: 27.1s	remaining: 2m 10s
172:	learn: 0.2984613	total: 27.3s	remaining: 2m 10s
173:	learn: 0.2983362	total: 27.4s	remaining: 2m 10s
174:	learn: 0.2981949	total: 27.6s	remaining: 2m 10s
175:	learn: 0.2978928	total: 27.7s	remaining: 2m 9

314:	learn: 0.2792537	total: 51.1s	remaining: 1m 51s
315:	learn: 0.2791419	total: 51.3s	remaining: 1m 50s
316:	learn: 0.2790344	total: 51.4s	remaining: 1m 50s
317:	learn: 0.2789004	total: 51.6s	remaining: 1m 50s
318:	learn: 0.2787980	total: 51.7s	remaining: 1m 50s
319:	learn: 0.2787540	total: 51.9s	remaining: 1m 50s
320:	learn: 0.2786385	total: 52.1s	remaining: 1m 50s
321:	learn: 0.2785141	total: 52.3s	remaining: 1m 50s
322:	learn: 0.2784155	total: 52.6s	remaining: 1m 50s
323:	learn: 0.2783231	total: 52.8s	remaining: 1m 50s
324:	learn: 0.2782544	total: 53s	remaining: 1m 50s
325:	learn: 0.2781853	total: 53.2s	remaining: 1m 49s
326:	learn: 0.2781359	total: 53.3s	remaining: 1m 49s
327:	learn: 0.2780507	total: 53.5s	remaining: 1m 49s
328:	learn: 0.2779604	total: 53.6s	remaining: 1m 49s
329:	learn: 0.2778167	total: 53.8s	remaining: 1m 49s
330:	learn: 0.2777213	total: 54s	remaining: 1m 49s
331:	learn: 0.2776795	total: 54.1s	remaining: 1m 48s
332:	learn: 0.2775572	total: 54.3s	remaining: 1m 4

470:	learn: 0.2677147	total: 1m 16s	remaining: 1m 25s
471:	learn: 0.2676031	total: 1m 16s	remaining: 1m 25s
472:	learn: 0.2675579	total: 1m 16s	remaining: 1m 25s
473:	learn: 0.2674952	total: 1m 16s	remaining: 1m 24s
474:	learn: 0.2674356	total: 1m 16s	remaining: 1m 24s
475:	learn: 0.2673729	total: 1m 16s	remaining: 1m 24s
476:	learn: 0.2673317	total: 1m 16s	remaining: 1m 24s
477:	learn: 0.2672949	total: 1m 17s	remaining: 1m 24s
478:	learn: 0.2672544	total: 1m 17s	remaining: 1m 24s
479:	learn: 0.2672203	total: 1m 17s	remaining: 1m 23s
480:	learn: 0.2671537	total: 1m 17s	remaining: 1m 23s
481:	learn: 0.2670903	total: 1m 17s	remaining: 1m 23s
482:	learn: 0.2670680	total: 1m 17s	remaining: 1m 23s
483:	learn: 0.2670076	total: 1m 18s	remaining: 1m 23s
484:	learn: 0.2669701	total: 1m 18s	remaining: 1m 23s
485:	learn: 0.2669521	total: 1m 18s	remaining: 1m 22s
486:	learn: 0.2669133	total: 1m 18s	remaining: 1m 22s
487:	learn: 0.2668060	total: 1m 18s	remaining: 1m 22s
488:	learn: 0.2667417	total:

In [82]:
company = 'ОАО "МЕСЯГУТОВСКИЙ КСОМ"'
extract_features(company, test=True)

array([[0.        , 0.04347826, 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.33333333, 0.1884058 , 0.61538462, 0.38461538, 0.        ,
        0.        , 0.04347826, 0.        , 1.        , 0.        ],
       [0.66666667, 0.05797101, 0.75      , 0.25      , 0.        ,
        0.33333333, 0.1884058 , 0.61538462, 0.38461538, 0.        ]])

In [83]:
transform(company, clf.predict(extract_features(company, test=True)))

[2.]
[1.]
[2.]


'ОАО "Месягутовский КСОМ"'

In [79]:
test_companies[1000:1200]

['КРЕСТЬЯНСКОЕ (ФЕРМЕРСКОЕ) ХОЗЯЙСТВО "ЮПИТЕР" БУРАЕВСКОГО РАЙОНА РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МКУ ОТДЕЛ ОБРАЗОВАНИЯ АМР БР РБ',
 'МОУ НОШ Д.НАРЫШЕВО',
 'МНОШ ДЕРЕВНИ ЯКУНИНО',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АИТОВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МУНИЦИПАЛЬНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ДЕТСКИЙ САД Д.КУЗБАЕВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МОБУ СОШ ИМ.М.Г.ХАЙРУЛЛИНОЙ Д.КАШКАЛЕВО',
 'МОБУ ООШ Д.МАМАДЫ',
 'МООШ Д.КАРГАЛЫ',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АСАВТАМАК" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МОБУ НОШ №3 С.БУРАЕВО',
 'МОБУ СОШ Д. БОЛЬШЕШУКШАНОВО',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ОСНОВНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д. БОЛЬШЕШУКШАНОВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВА

In [29]:
result = open('result.txt', 'w')
for company in test_companies:
    result.write(transform(company, clf.predict(extract_features(company, test=True))) + '\n')

In [12]:
a = [[1], [2], [3]]
b = [[1], [2], [3]]
b + a

[[1], [2], [3], [1], [2], [3]]